In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv(".env")

True

In [3]:
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version=os.getenv("AZURE_OPENAI_VERSION")
)
model = os.getenv("AZURE_OPENAI_NAME")

In [4]:
# BASIC
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
    model=model,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":text_prompt},])

response.choices[0].message.content

'The use of the Oxford comma (also known as the serial comma) is a matter of style, and whether or not it should always be used depends on the style guide or personal preference you follow. An Oxford comma is the final comma in a list of three or more items, placed before the word "and" or "or." For example:\n\n- With Oxford comma: I love my parents, Taylor Swift, and pizza.\n- Without Oxford comma: I love my parents, Taylor Swift and pizza.\n\nThe Oxford comma is often recommended because it can reduce ambiguity. For instance:\n\n- Without Oxford comma: I’d like to thank my parents, Oprah Winfrey and God. (This implies the parents are Oprah Winfrey and God.)\n- With Oxford comma: I’d like to thank my parents, Oprah Winfrey, and God. (Clearly separates into three distinct entities.)\n\n### When to Use the Oxford Comma\n1. **Clarity**: Use an Oxford comma if omitting it would create confusion or ambiguity.\n2. **Style Guides**: Certain style guides, such as **The Chicago Manual of Style

In large language models (LLMs), the terms "system," "user," and "assistant" refer to different roles within a conversation. "System" provides instructions or sets the context for the LLM, "user" represents the user's input, and "assistant" is the LLM's response. 

**Elaboration:**
- System: This role is used to provide instructions, guidelines, or context for the LLM's behavior. It's often used to define the LLM's personality, style, or desired output format.
- User: This role represents the user's input, which can be a question, command, or any other kind of prompt. It's the starting point for the LLM to generate a response.
- Assistant: This role represents the LLM's response to the user's input. It's the output generated by the model based on the instructions and user input.

All parameters explained:
1. **temperature**: Controls the randomness of the output. A value of 0 makes the output more deterministic. (Higher values increase creativity)
2. **max_tokens**: The maximum number of tokens (length) in the generated output.
3. **top_p**: Controls diversity via nucleus sampling. (Limits token selection to the most probable options, whose cumulative probability meets a specified threshold, balancing diversity and coherence.) Lower values result in high-probability tokens, leading to more predictable and focused outputs. This is beneficial for tasks requiring precision and factual accuracy.
4. **frequency_penalty**: Reduces the likelihood of repeating words or phrases by penalizing frequently used tokens.
5. **presence_penalty**:  Promotes novelty by penalizing tokens that have already appeared.
6. **stop**: Specifies stopping sequences to end the generated text appropriately.

Considerations
- temperature and top_p are functionally similar, hence it is recommended to tune either one of them and NOT both.
- max_tokens include both the input tokens and the output tokens. Use this to generate a shorter response or lower api cost. (< ~200 token)
- frequency_penalty had a range of -2.0 and 2.0. Negative values encourage the reuse of tokens, increasing repetition.
- presence_penalty had a range of -2.0 and 2.0. It penalizes any token that has already been used in the output (even if it appeared only once)

There are three basic guidelines to creating prompts:

Show and tell. Make it clear what you want either through instructions, examples, or a combination of the two. If you want the model to rank a list of items in alphabetical order or to classify a paragraph by sentiment, show it that's what you want.

Provide quality data. If you're trying to build a classifier or get the model to follow a pattern, make sure that there are enough examples. Be sure to proofread your examples — the model is usually smart enough to see through basic spelling mistakes and give you a response, but it also might assume this is intentional and it can affect the response.

Check your settings. The temperature and top_p settings control how deterministic the model is in generating a response. If you're asking it for a response where there's only one right answer, then you'd want to set these lower. If you're looking for more diverse responses, then you might want to set them higher. The number one mistake people use with these settings is assuming that they're "cleverness" or "creativity" controls.

- add `Tl;dr` to summarize text

In [5]:
prompt = "### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\n query: "
print(prompt)
response = client.chat.completions.create(
  model = model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": prompt}],
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response.choices[0].message.content)

### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months

 query: 
To list the names of the departments that employed more than 10 employees in the last 3 months, you can use the following SQL query:

```sql
SELECT d.name AS department_name
FROM Department d
JOIN Employee e ON d.id = e.department_id
WHERE e.id IN (
    SELECT DISTINCT employee_id
    FROM Salary_Payments
    WHERE date >= CURRENT_DATE - INTERVAL '3 months'
)
GROUP BY d.id, d.name
HAVING COUNT(e.id) > 10


In [ ]:
# See token usage
response.usage

CompletionUsage(completion_tokens=103, prompt_tokens=87, total_tokens=190, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [ ]:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython